In [365]:
import pandas as pd

In [366]:
teams = pd.read_csv("../data/MTeams.csv")

In [367]:
slots = pd.read_csv("../data/MNCAATourneySlots.csv")
slots = slots[slots.Season == 2023]

In [368]:
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

# remove play-ins
seeds["Seed"] = seeds["Seed"].str[0:3]
seeds = seeds.drop_duplicates(["Season", "Seed"])
seeds["Seed"] = seeds["Seed"].astype(str)

In [369]:
predictions = pd.read_csv("../submissions/sub1_3_13.csv")
predictions[['Season', 'Team1', 'Team2']] = predictions['ID'].str.split('_', expand=True)
predictions = predictions[predictions["Pred"] != .5] 
predictions["Team1"] = predictions["Team1"].astype(int) 
predictions["Team2"] = predictions["Team2"].astype(int) 
predictions["Season"] = predictions["Season"].astype(int) 

seeds_t1 = seeds.copy()
seeds_t1.columns = ["Season", "t1_seed", "Team1"]
predictions = predictions.merge(seeds_t1, how="inner", on=["Season", "Team1"]) # inner results in removing playin teams from above 

seeds_t2 = seeds.copy()
seeds_t2.columns = ["Season", "t2_seed", "Team2"]
predictions = predictions.merge(seeds_t2, how="inner", on=["Season", "Team2"])

# filter out women for now
predictions = predictions[predictions.Team1 < 2000]

In [370]:
predictions2 = predictions.copy()
predictions2 = predictions[['ID', 'Pred', 'Season', 'Team2', 'Team1', 't2_seed', 't1_seed']]
predictions2.columns = ['ID', 'Pred', 'Season', 'Team1', 'Team2', 't1_seed', 't2_seed']
predictions2["Pred"] = 1 - predictions2["Pred"]
predictions = pd.concat([predictions, predictions2], axis=0)

In [371]:
# import re
# predictions["t1_seed_num"] = predictions['t1_seed'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else None)
# predictions["t2_seed_num"] = predictions['t2_seed'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else None)

# import numpy as np
# predictions["StrongSeed"] = np.where(predictions.t1_seed_num < predictions.t2_seed_num,
#                                     predictions.t1_seed,  predictions.t2_seed)

# predictions["WeakSeed"] = np.where(predictions.t1_seed_num < predictions.t2_seed_num,
#                                     predictions.t2_seed,  predictions.t1_seed)  

# predictions["prob_strong_seed"] = np.where(predictions.t1_seed_num < predictions.t2_seed_num,
#                                     predictions["Pred"],  1 - predictions["Pred"])

                           

In [372]:
def get_round(slots, round):
    return slots[slots.Slot.str[1] == str(round)]

r1_slots = get_round(slots, 1)
r2_slots = get_round(slots, 2)
r3_slots = get_round(slots, 3)
r4_slots = get_round(slots, 4)
r5_slots = get_round(slots, 5)
r6_slots = get_round(slots, 6)


In [373]:
predictions_copy = predictions.copy()
predictions_copy.columns = ['ID', 'prob_strong_seed', 'Season', 'StrongTeam', 'WeakTeam', 'StrongSeed', 'WeakSeed']

In [374]:
import numpy as np
def random_choice(p):
    return np.random.choice([1, 0], p=[p, 1 - p])


In [375]:
# get probs
r1_slots = r1_slots.merge(predictions_copy, how="left", on = ["Season", "StrongSeed", "WeakSeed"])
r1_slots = r1_slots[r1_slots.prob_strong_seed.notnull()]

# Get whether strong seed wins
r1_slots['strong_seed_wins'] = r1_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

# get slot / winners
r1_slots["winner_team"] = np.where(r1_slots.strong_seed_wins == 1, r1_slots.StrongTeam, r1_slots.WeakTeam)
r1_winners = r1_slots[["Slot", "winner_team"]]




In [376]:
r1_winners1 = r1_winners.copy()
r1_winners1.columns = ["StrongSeed", "StrongTeam"]

r2_slots = r2_slots.merge(r1_winners1, how="left", on=["StrongSeed"])

r1_winners2 = r1_winners.copy()
r1_winners2.columns = ["WeakSeed", "WeakTeam"]

r2_slots = r2_slots.merge(r1_winners2, how="left", on=["WeakSeed"])

key = ["Season", "StrongTeam", "WeakTeam"]
r2_slots = r2_slots.merge(predictions_copy[key +["prob_strong_seed"]], how="left", on = key)

r2_slots['strong_seed_wins'] = r2_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

r2_slots["winner_team"] = np.where(r2_slots.strong_seed_wins == 1, r2_slots.StrongTeam, r2_slots.WeakTeam)
r2_winners = r2_slots[["Slot", "winner_team"]]



In [377]:
# merge in winners
r2_winners1 = r2_winners.copy()
r2_winners1.columns = ["StrongSeed", "StrongTeam"]

r3_slots = r3_slots.merge(r2_winners1, how="left", on=["StrongSeed"])

r2_winners2 = r2_winners.copy()
r2_winners2.columns = ["WeakSeed", "WeakTeam"]

r3_slots = r3_slots.merge(r2_winners2, how="left", on=["WeakSeed"])

key = ["Season", "StrongTeam", "WeakTeam"]
r3_slots = r3_slots.merge(predictions_copy[key +["prob_strong_seed"]], how="left", on = key)

r3_slots['strong_seed_wins'] = r3_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

r3_slots["winner_team"] = np.where(r3_slots.strong_seed_wins == 1, r3_slots.StrongTeam, r3_slots.WeakTeam)
r3_winners = r3_slots[["Slot", "winner_team"]]

In [ ]:
r3_winners.merge(teams, how="left", left_on="winner_team",right_on="TeamID")

In [378]:
# merge in winners
r3_winners1 = r3_winners.copy()
r3_winners1.columns = ["StrongSeed", "StrongTeam"]

r4_slots = r4_slots.merge(r3_winners1, how="left", on=["StrongSeed"])

r3_winners2 = r3_winners.copy()
r3_winners2.columns = ["WeakSeed", "WeakTeam"]

r4_slots = r4_slots.merge(r3_winners2, how="left", on=["WeakSeed"])

key = ["Season", "StrongTeam", "WeakTeam"]
r4_slots = r4_slots.merge(predictions_copy[key +["prob_strong_seed"]], how="left", on = key)

r4_slots['strong_seed_wins'] = r4_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

r4_slots["winner_team"] = np.where(r4_slots.strong_seed_wins == 1, r4_slots.StrongTeam, r4_slots.WeakTeam)
r4_winners = r4_slots[["Slot", "winner_team"]]

In [ ]:
r4_winners.merge(teams, how="left", left_on="winner_team",right_on="TeamID")

In [379]:
# merge in winners
r4_winners1 = r4_winners.copy()
r4_winners1.columns = ["StrongSeed", "StrongTeam"]

r5_slots = r5_slots.merge(r4_winners1, how="left", on=["StrongSeed"])

r4_winners2 = r4_winners.copy()
r4_winners2.columns = ["WeakSeed", "WeakTeam"]

r5_slots = r5_slots.merge(r4_winners2, how="left", on=["WeakSeed"])

key = ["Season", "StrongTeam", "WeakTeam"]
r5_slots = r5_slots.merge(predictions_copy[key +["prob_strong_seed"]], how="left", on = key)

r5_slots['strong_seed_wins'] = r5_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

r5_slots["winner_team"] = np.where(r5_slots.strong_seed_wins == 1, r5_slots.StrongTeam, r5_slots.WeakTeam)
r5_winners = r5_slots[["Slot", "winner_team"]]

In [ ]:
r5_winners.merge(teams, how="left", left_on="winner_team",right_on="TeamID")

In [380]:
# merge in winners
r5_winners1 = r5_winners.copy()
r5_winners1.columns = ["StrongSeed", "StrongTeam"]

r6_slots = r6_slots.merge(r5_winners1, how="left", on=["StrongSeed"])

r5_winners2 = r5_winners.copy()
r5_winners2.columns = ["WeakSeed", "WeakTeam"]

r6_slots = r6_slots.merge(r5_winners2, how="left", on=["WeakSeed"])

key = ["Season", "StrongTeam", "WeakTeam"]
r6_slots = r6_slots.merge(predictions_copy[key +["prob_strong_seed"]], how="left", on = key)

r6_slots['strong_seed_wins'] = r6_slots['prob_strong_seed'].apply(lambda x: random_choice(x))

r6_slots["winner_team"] = np.where(r6_slots.strong_seed_wins == 1, r6_slots.StrongTeam, r6_slots.WeakTeam)
r6_winners = r6_slots[["Slot", "winner_team"]]

In [381]:
r6_winners.merge(teams, how="left", left_on="winner_team",right_on="TeamID")

,Slot,winner_team,TeamID,TeamName,FirstD1Season,LastD1Season
0,R6CH,1104.0,1104,Alabama,1985,2024
